## Linear Programming, LP

In [9]:
import pandas as pd
import pulp

# === 1. Read data ===
forwarder_file = r"\Users\Sample Project files\Sample Project files\telemetry-main\data\Appendix 2 Data of 8 forwarders in the past 5 years.xlsx"
loss_df = pd.read_excel(forwarder_file, sheet_name='Loss rate')
weekly_plan = pd.read_excel("24_week_order_plan.xlsx", index_col=0)

# === 2. Extract basic information ===
forwarders = loss_df.iloc[:, 0].tolist()  # Forwarder names
loss_rate = loss_df.select_dtypes(include='number').mean(axis=1).tolist()  # Average loss rate for each forwarder
loss_rate_dict = dict(zip(forwarders, loss_rate))  # Convert to dictionary for easy lookup

suppliers = weekly_plan.index.tolist()
weeks = weekly_plan.columns.tolist()

# === 3. Model parameters ===
capacity = 6000  # Maximum transport capacity per forwarder per week

# === 4. Create linear programming model ===
prob = pulp.LpProblem("Forwarder_Allocation", pulp.LpMinimize)

# Decision variable x[i, j, t]: Whether supplier i chooses forwarder j in week t (0/1)
x = pulp.LpVariable.dicts(
    "x",
    ((i, j, t) for i in suppliers for j in forwarders for t in weeks),
    cat='Binary'
)

# === 5. Objective function: Minimize total loss (order volume × loss rate) ===
prob += pulp.lpSum(
    x[i, j, t] * loss_rate_dict[j] * weekly_plan.loc[i, t]
    for i in suppliers for j in forwarders for t in weeks
)

# === 6. Constraint: Each supplier can only be assigned to one forwarder per week ===
for i in suppliers:
    for t in weeks:
        if weekly_plan.loc[i, t] > 0:
            prob += pulp.lpSum(x[i, j, t] for j in forwarders) == 1
        else:
            prob += pulp.lpSum(x[i, j, t] for j in forwarders) == 0

# === 7. Constraint: Each forwarder's total transport volume per week cannot exceed 6000 ===
for j in forwarders:
    for t in weeks:
        prob += pulp.lpSum(x[i, j, t] * weekly_plan.loc[i, t] for i in suppliers) <= capacity

# === 8. Solve ===
prob.solve()

# === 9. Check solution status ===
if pulp.LpStatus[prob.status] != 'Optimal':
    print("⚠️ Solution did not reach an optimal state, status:", pulp.LpStatus[prob.status])
else:
    print("✅ Solution successful, status:", pulp.LpStatus[prob.status])

# === 10. Build result table ===
transfer_plan = pd.DataFrame(index=suppliers, columns=weeks)
for i in suppliers:
    for t in weeks:
        if weekly_plan.loc[i, t] > 0:
            assigned = False
            for j in forwarders:
                if pulp.value(x[i, j, t]) == 1:
                    transfer_plan.loc[i, t] = j
                    assigned = True
                    break
            if not assigned:
                transfer_plan.loc[i, t] = "No Capacity"
        else:
            transfer_plan.loc[i, t] = "No Order"

# === 11. Save results ===
transfer_plan.to_excel("24_week_forwarder_plan_LP.xlsx")
print("📁 Transfer allocation completed, results saved to: 24_week_forwarder_plan_LP.xlsx")

⚠️ Solution did not reach an optimal state, status: Infeasible
📁 Transfer allocation completed, results saved to: 24_week_forwarder_plan_LP.xlsx


## Random Forest

In [8]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# === 1. Read data ===
forwarder_file = r"\Users\Sample Project files\Sample Project files\telemetry-main\data\Appendix 2 Data of 8 forwarders in the past 5 years.xlsx"
loss_df = pd.read_excel(forwarder_file, sheet_name='Loss rate')
weekly_plan = pd.read_excel("24_week_order_plan.xlsx", index_col=0)

# Extract forwarders and average loss rates
forwarders = loss_df.iloc[:, 0].tolist()
loss_rate = loss_df.select_dtypes(include='number').mean(axis=1).tolist()

# Extract suppliers and weeks
suppliers = weekly_plan.index.tolist()
weeks = weekly_plan.columns.tolist()

# === 2. Build training data (generate random historical records) ===
features = []
labels = []

for supplier in suppliers:
    for week in weeks:
        order_volume = weekly_plan.loc[supplier, week]
        if order_volume > 0:
            fwd = random.choice(forwarders)  # Simulate historical records
            features.append([order_volume, loss_rate[forwarders.index(fwd)]])
            labels.append(fwd)
        else:
            features.append([0, 0])
            labels.append("No Order")

# Convert to DataFrame
features_df = pd.DataFrame(features, columns=['OrderVolume', 'LossRate'])
labels_df = pd.Series(labels, name='Forwarder')

# === 3. Model training ===
X_train, X_test, y_train, y_test = train_test_split(features_df, labels_df, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Model accuracy:", accuracy_score(y_test, y_pred))

# === 4. Construct future prediction plan + limit each forwarder's weekly capacity to 6000 ===
transfer_plan = pd.DataFrame(index=suppliers, columns=weeks)
weekly_forwarder_load = pd.DataFrame(0, index=weeks, columns=forwarders)  # Weekly cumulative transport volume for each forwarder

for supplier in suppliers:
    for week in weeks:
        order_volume = weekly_plan.loc[supplier, week]
        if order_volume > 0:
            # Identify forwarders that have not exceeded their limit this week
            valid_forwarders = [
                fwd for fwd in forwarders
                if weekly_forwarder_load.loc[week, fwd] + order_volume <= 6000
            ]

            if valid_forwarders:
                # Use the model to predict probabilities for all candidates
                probs = clf.predict_proba([[order_volume, min(loss_rate)]])[0]
                label_indices = clf.classes_
                fwd_scores = {
                    fwd: probs[i]
                    for i, fwd in enumerate(label_indices)
                    if fwd in valid_forwarders
                }

                best_fwd = max(fwd_scores, key=fwd_scores.get)
                transfer_plan.loc[supplier, week] = best_fwd
                weekly_forwarder_load.loc[week, best_fwd] += order_volume
            else:
                transfer_plan.loc[supplier, week] = "No Available Forwarder"
        else:
            transfer_plan.loc[supplier, week] = "No Order"

# === 5. Save results ===
transfer_plan.to_excel("24_week_forwarder_plan_ML_with_limit.xlsx")
weekly_forwarder_load.to_excel("weekly_forwarder_load_report.xlsx")
print("Transfer allocation completed and saved to: 24_week_forwarder_plan_ML_with_limit.xlsx")
print("Weekly forwarder load report saved to: weekly_forwarder_load_report.xlsx")

Model accuracy: 1.0


C:\Users\12620\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Windows\Temp\ipykernel_27028\2161358965.py:75: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8374206256898079' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weekly_forwarder_load.loc[week, best_fwd] += order_volume
C:\Users\12620\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\12620\AppData\Local\Packages\PythonSoftwareFoundati

Transfer allocation completed and saved to: 24_week_forwarder_plan_ML_with_limit.xlsx
Weekly forwarder load report saved to: weekly_forwarder_load_report.xlsx
